Atividade para a conclusão do curso "Big Data e Astroinformática"

Prof. Clécio de Bom

Aluno Maurício Marques Soares Filho




---



Part I: Introduction

In [16]:
# 12/12/2021
# Designed on Google Colab

# Import necessary packages
from astropy.io import fits
from astropy.table import Table
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM

# if necessary in the future...
import matplotlib.pyplot as plt     
%matplotlib inline                  
import seaborn                      

# Ignoring warnings...
import warnings
warnings.filterwarnings("ignore")

# Download the .fits file
!wget -O catalogo01.fits "https://www.dropbox.com/s/nc162bxppd7qmjn/catalogo01?dl=0"
!ls

# Open catalog from fits table using astropy package
hdul = fits.open("catalogo01.fits", ignore_missing_end=True) 
data = hdul[1].data # table extensions can't be the first extension, so there's a dummy image extension at 0
cat_table = Table(data)
cols=cat_table.columns
print(cols)

# Constructing Pandas DataFrame from Table fits
df = cat_table.to_pandas()

# Checking for NaN entries
check_for_nan = df.isnull().any().any()
if check_for_nan == True:
  df.dropna()
  print("Lines with NaN values eliminated!")

Nrows = len(df)
print(df)

# Device to save outlier positions
num_index = []

--2021-12-16 20:55:07--  https://www.dropbox.com/s/nc162bxppd7qmjn/catalogo01?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nc162bxppd7qmjn/catalogo01 [following]
--2021-12-16 20:55:08--  https://www.dropbox.com/s/raw/nc162bxppd7qmjn/catalogo01
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7ad5356b9e77c9034e257df954.dl.dropboxusercontent.com/cd/0/inline/Bb_NLLPjIndylGW6WMW_SFB6A7e-C-_jpUndqBs1HTT41SGUZ3ItJkGkKrHwwufxPnNTaL0ZOK5e7PS3DW57Ybo0BXrMwFtjbPqpcLflqGnMvDhOnB_bnHcYPL6EQ5HhGSSkdJ0BNwBVKa-gP5PoZRyK/file# [following]
--2021-12-16 20:55:08--  https://uc7ad5356b9e77c9034e257df954.dl.dropboxusercontent.com/cd/0/inline/Bb_NLLPjIndylGW6WMW_SFB6A7e-C-_jpUndqBs1HTT41SGUZ3ItJkGkKrHwwufxPnNTaL0ZOK5e7PS3DW57Ybo0

Part II: Linear Regression and Search for Mean Absolute Error (MAE)

In [17]:
#Regression predictive modeling problem, preparation

# retrieve the array
data = df.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the dataset
print(X.shape, y.shape)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=5)
# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(100000, 7) (100000,)
(67000, 7) (33000, 7) (67000,) (33000,)
MAE: 0.775


Part III - Search for Outliers

Method 1 - Isolation Forest

In [18]:
# evaluate model performance with outliers removed using Isolation Forest

# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
mod = y_train.shape
# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)
print("Outliers quantity: %.0f" % (np.array(mod)-np.array(y_train.shape)))
# finding the most eccentric outliers, training set and test set
tmp = iso.score_samples(X_train)
ind_tr = np.where(tmp == tmp.min())
row_tr = X_train[ind_tr].ravel()
for i in range (0, (Nrows-1)):
  if set(row_tr).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Train Set = %.0f" % i)
    num_index.append(i)
    break
tmp = iso.score_samples(X_test)
ind_ts = np.where(tmp == tmp.min())
row_ts = X_test[ind_ts].ravel()
for i in range (0, (Nrows-1)):
  if set(row_ts).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Test Set = %.0f" % i)
    num_index.append(i)
    break

(67000, 7) (67000,)
(60300, 7) (60300,)
MAE: 0.786
Outliers quantity: 6700
Row Index, Max Outlier in Train Set = 9994
Row Index, Max Outlier in Test Set = 51841


Method 2 - Minimum Covariance Determinant

In [19]:
# evaluate model performance with outliers removed using Minimum Covariance Determinant/Elliptical Envelope

# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
mod = y_train.shape
# identify outliers in the training dataset
ee = EllipticEnvelope(contamination=0.01)
yhat = ee.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)
print("Outliers quantity: %.0f" % (np.array(mod)-np.array(y_train.shape)))
# finding the most eccentric outliers, training set and test set
tmp = ee.score_samples(X_train)
ind_tr = np.where(tmp == tmp.min())
row_tr = X_train[ind_tr].ravel()
for i in range (0, (Nrows-1)):
  if set(row_tr).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Train Set = %.0f" % i)
    num_index.append(i)
    break
tmp = ee.score_samples(X_test)
ind_ts = np.where(tmp == tmp.min())
row_ts = X_test[ind_ts].ravel()
for i in range (0, (Nrows-1)):
  if set(row_ts).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Test Set = %.0f" % i)
    num_index.append(i)
    break

(67000, 7) (67000,)
(66330, 7) (66330,)
MAE: 0.699
Outliers quantity: 670
Row Index, Max Outlier in Train Set = 51997
Row Index, Max Outlier in Test Set = 36521


Method 3 - Local Outlier Factor

In [20]:
# evaluate model performance with outliers removed using Local Outlier Factor

# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)
print("Outliers quantity: %.0f" % (np.array(mod)-np.array(y_train.shape)))
# finding the most eccentric outlier (only possible for the training set in this case)
tmp = lof.negative_outlier_factor_
ind_tr = np.where(tmp == tmp.min())
row_tr = X_train[ind_tr].ravel()
for i in range (0, (Nrows-1)):
  if set(row_tr).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Train Set = %.0f" % i)
    num_index.append(i)
    break

(67000, 7) (67000,)
(64650, 7) (64650,)
MAE: 0.758
Outliers quantity: 2350
Row Index, Max Outlier in Train Set = 84877


Method 4 - One-Class SVM

In [21]:
# evaluate model performance with outliers removed using One Class SVM

# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
mod = y_train.shape
# identify outliers in the training dataset
ee = OneClassSVM(nu=0.01)
yhat = ee.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)
print("Outliers quantity: %.0f" % (np.array(mod)-np.array(y_train.shape)))
# finding the most eccentric outliers, training set and test set
tmp = ee.score_samples(X_train)
ind_tr = np.where(tmp == tmp.min())
row_tr = X_train[ind_tr].ravel()
for i in range (0, (Nrows-1)):
  if set(row_tr).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Train Set = %.0f" % i)
    num_index.append(i)
    break
tmp = ee.score_samples(X_test)
ind_ts = np.where(tmp == tmp.min())
row_ts = X_test[ind_ts].ravel()
for i in range (0, (Nrows-1)):
  if set(row_ts).issubset(set(df.loc[i])):
    print("Row Index, Max Outlier in Test Set = %.0f" % i)
    num_index.append(i)
    break

(67000, 7) (67000,)
(66331, 7) (66331,)
MAE: 0.731
Outliers quantity: 669
Row Index, Max Outlier in Train Set = 51273
Row Index, Max Outlier in Test Set = 34225


Part IV: Saving the Results

In [22]:
# Download the .fits file adapted for TOPCAT and ALADIN
!wget -O catalogo02.fits "https://www.dropbox.com/s/si2mvowfmpji22r/catalogo02.fits?dl=0"
!ls

hdul = fits.open("catalogo02.fits", ignore_missing_end=True) 
data = hdul[1].data # table extensions can't be the first extension, so there's a dummy image extension at 0
cat_table = Table(data)
cols=cat_table.columns

df01 = cat_table.to_pandas()

df02 = pd.DataFrame(columns=cols)

for j in num_index:
  df02 = df02.append(df01.loc[j], ignore_index=True)

# Saving the data in your Google Drive, at Results.csv file
from google.colab import drive
drive.mount('/content/drive')
df02.to_csv('/content/drive/MyDrive/Results.csv', index=False)

print("THE END")

--2021-12-16 20:56:46--  https://www.dropbox.com/s/si2mvowfmpji22r/catalogo02.fits?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/si2mvowfmpji22r/catalogo02.fits [following]
--2021-12-16 20:56:47--  https://www.dropbox.com/s/raw/si2mvowfmpji22r/catalogo02.fits
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc14ecec06d121d2a7416088931a.dl.dropboxusercontent.com/cd/0/inline/Bb-aIT-PSp8DpIkdIFEEngNA6Y1oOG8tZ4KQXCycYDxuvqp27Z-L06MFtxsQZRweYce0sF07tF5mSTqXdiw4q0Gznsa2QzEA3-wUU6CEnVsvJUhZuydOo_WI-vpXvleCi9x7dbDT-movF07G2SfuC82Z/file# [following]
--2021-12-16 20:56:47--  https://uc14ecec06d121d2a7416088931a.dl.dropboxusercontent.com/cd/0/inline/Bb-aIT-PSp8DpIkdIFEEngNA6Y1oOG8tZ4KQXCycYDxuvqp27Z-L06MFtxsQZRweYce0sF07t